# Ranking

Second Assignement of Learning with Massive Data. 


# Usefull Function

In [1]:
import pandas as pd

def score_table(score):
    df = pd.DataFrame(score)

    df.index = ["Query " + str(i+1) for i in range(len(df.index))]
    df.columns = ["Document " + str(i+1) for i in range(len(df.columns))]

    return df


In [2]:
import json
import numpy as np
import string

path_name_documents = './Databases/prova/gigi.jsonl'
path_name_query = './Databases/prova/query.jsonl'


def readFile(path_name):
    # Load the JSONL file into a list
    with open(path_name, 'r') as f:
        lines = f.readlines()

    # Convert each JSON object into a dictionary
    dicts = [json.loads(line) for line in lines]

    # Convert the dictionaries into arrays and stack them vertically
    arrays = np.vstack([np.array(list(d.values())) for d in dicts])

    # Convert the arrays into a list of lists
    text = arrays.tolist()
    
    return text

documents = readFile(path_name_documents)
queries = readFile(path_name_query)


# Tokenize

In [3]:
import json
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer


stop_words = set(stopwords.words('english'))

def stemmingLemming(filtered_tokens):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    # Perform stemming or lemmatization on filtered tokens
    
    filtered_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    filtered_tokens = [stemmer.stem(token) for token in filtered_tokens]

    return filtered_tokens
    
 
    

def tokenize(path_name):
    
    with open(path_name, "r") as f:
        data = f.readlines()

        # Create an empty list to store the tokenized documents
        tokenized_docs = []

        # Loop through each line in the JSONL file
        for line in data:
            # Parse the JSON string into a Python dictionary
            doc = json.loads(line)

            # Extract the text from the dictionary
            text = doc['text']
            text = text.lower()  # Convert to lowercase
            #text = re.sub(r'\d+', '', text)  # Remove all numbers
            text = text.translate(str.maketrans('', '', string.punctuation))  # Remove all punctuation

            # Tokenize the text using NLTK
            tokens = word_tokenize(text)
            tokensStemLem = stemmingLemming(tokens)

            # Add the tokenized document to the list
            tokenized_docs.append(tokensStemLem)

        # Print the tokenized documents
    return tokenized_docs

tokenized_docs = tokenize(path_name_documents)


tokenized_query = tokenize(path_name_query)




# Sparse Vectors



# TF-IDF 

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

def calculateTFIDF(tokenized_docs):
    # Initialize the TfidfVectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the tokenized documents into a TF-IDF matrix
    tfidf_matrix = vectorizer.fit_transform([' '.join(doc) for doc in tokenized_docs])

    # Get the feature names (tokens)
    feature_names = vectorizer.get_feature_names()

    # Return the TF-IDF matrix and the feature names
    return tfidf_matrix, feature_names,vectorizer
 
    
    
def calculateTFIDFQuery(tokenized_query,vectorizer):
    
    # Fit and transform the tokenized documents into a TF-IDF matrix
    tfidf_matrix = vectorizer.transform([' '.join(doc) for doc in tokenized_query])

    # Get the feature names (tokens)
    feature_names = vectorizer.get_feature_names()

    # Return the TF-IDF matrix and the feature names
    return tfidf_matrix, feature_names

    

tfidf_matrix_docs, feature_names_docs,vectorizer  = calculateTFIDF(tokenized_docs)
tfidf_matrix_query, feature_names_query  = calculateTFIDFQuery( tokenized_query,vectorizer)


C:\Users\lita4\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# Dot Product for Sparse Vectors

In [5]:
import numpy as np


def calculate_Dot_Product_Sparse(tfidf_matrix_docs, tfidf_matrix_query):
    matrix_docs = tfidf_matrix_docs.toarray()
    matrix_query = tfidf_matrix_query.toarray()
    num_queries = matrix_query.shape[0]
    num_docs = matrix_docs.shape[0]
    sparse_score_results = []
    
    for query in range(num_queries):
        sparse_score_docs = []
        for doc in range(num_docs):
            dot_result = np.dot(matrix_query[query], matrix_docs[doc])
            sparse_score_docs.append(dot_result)
        sparse_score_results.append(sparse_score_docs)
        
    return sparse_score_results



sparse_score_results = calculate_Dot_Product_Sparse(tfidf_matrix_docs,tfidf_matrix_query)

print(sparse_score_results)

sparse_score_table = score_table(sparse_score_results)
sparse_score_table
    


[[0.18814208644069141, 0.33933990533577935, 0.13560874763494904, 0.3664275073457122, 0.37700842205641716, 0.21497994496187967, 0.16838125469128457, 0.218125078598137, 0.0975490133688652, 0.3349435769159468, 0.18814208644069141], [0.22491358017656699, 0.1492127993994314, 0.10267125124219378, 0.19910949239594666, 0.2256525556007502, 0.07907883981614139, 0.17152278627797812, 0.14768053202783524, 0.2294239320380127, 0.15781821356506004, 0.22491358017656699], [0.037955266788926634, 0.015821364844910025, 0.09098290919785915, 0.018306756488451155, 0.09411624307911748, 0.0578772033840427, 0.0942715213793465, 0.006930357337788915, 0.027940751768459223, 0.016733816040142315, 0.037955266788926634], [0.021957283860775365, 0.0, 0.026377225692876035, 0.0, 0.016987194899028817, 0.0, 0.08904339776606349, 0.052128511810284994, 0.0, 0.0, 0.021957283860775365], [0.0612672909000269, 0.015065049582750737, 0.010017452484352348, 0.05272447083422632, 0.04870300863824418, 0.12134277312006807, 0.018657548803557

,Document 1,Document 2,Document 3,Document 4,Document 5,Document 6,Document 7,Document 8,Document 9,Document 10,Document 11
Query 1,0.188142,0.339340,0.135609,0.366428,0.377008,0.214980,0.168381,0.218125,0.097549,0.334944,0.188142
Query 2,0.224914,0.149213,0.102671,0.199109,0.225653,0.079079,0.171523,0.147681,0.229424,0.157818,0.224914
Query 3,0.037955,0.015821,0.090983,0.018307,0.094116,0.057877,0.094272,0.006930,0.027941,0.016734,0.037955
Query 4,0.021957,0.000000,0.026377,0.000000,0.016987,0.000000,0.089043,0.052129,0.000000,0.000000,0.021957
Query 5,0.061267,0.015065,0.010017,0.052724,0.048703,0.121343,0.018658,0.040877,0.083512,0.015934,0.061267
Query 6,0.074024,0.062264,0.034060,0.096203,0.083014,0.101766,0.036720,0.044875,0.055716,0.056447,0.074024
Query 7,0.076579,0.016662,0.030792,0.115323,0.130225,0.035322,0.159456,0.093255,0.137412,0.017623,0.076579
Query 8,0.207297,0.220873,0.101320,0.287763,0.256384,0.187223,0.173456,0.141129,0.131020,0.200238,0.207297


# Dense Vector

In [6]:

from sentence_transformers import SentenceTransformer

def create_dense_vector(document):
    # Load the all-MiniLM-L6-v2 model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Create a dense vector representation of the document
    dense_vector = model.encode(document, convert_to_tensor=True)
    
    
    return dense_vector


dense_vector_document = create_dense_vector(documents)
dense_vector_query = create_dense_vector(queries)




# Dot Product for Dense Vector

In [7]:
def calculate_Dot_Product_Dense(dense_vector_document, dense_vector_query):
    num_queries = dense_vector_query.shape[0]
    num_docs = dense_vector_document.shape[0]
    dense_score_results = []
    
    for query in range(num_queries):
        dense_score_docs = []
        for doc in range(num_docs):
            dot_result = np.dot(dense_vector_query[query], dense_vector_document[doc])
            dense_score_docs.append(dot_result)
        dense_score_results.append(dense_score_docs)
        
    return dense_score_results


dense_score_results = calculate_Dot_Product_Dense(dense_vector_document, dense_vector_query)

dense_score_table = score_table(dense_score_results)
dense_score_table
    

,Document 1,Document 2,Document 3,Document 4,Document 5,Document 6,Document 7,Document 8,Document 9,Document 10,Document 11
Query 1,0.077958,0.038245,0.133962,0.106983,0.226969,0.126819,0.324864,0.217792,0.038656,0.038245,0.077958
Query 2,0.099590,0.070468,0.066888,0.096966,0.279706,0.121130,0.310443,0.200889,0.052011,0.070468,0.099590
Query 3,0.115690,0.103854,0.194563,0.120762,0.322730,0.129222,0.261768,0.200194,0.167548,0.103854,0.115690
Query 4,0.144724,0.175452,0.131543,0.226517,0.249557,0.214995,0.339539,0.206882,0.168591,0.175452,0.144724
Query 5,0.229337,0.190014,0.095667,0.222562,0.330017,0.203857,0.242839,0.142185,0.110872,0.190014,0.229337
Query 6,0.092435,0.147272,0.173545,0.096595,0.171428,0.057788,0.219801,0.182564,0.211420,0.147272,0.092435
Query 7,0.174007,0.177980,0.213392,0.104680,0.231447,0.153244,0.285167,0.150482,0.233630,0.177980,0.174007
Query 8,0.129492,0.135007,0.152630,0.118768,0.170204,0.102133,0.181831,0.162573,0.278159,0.135007,0.129492


# ----------------------------------------------------------

# Calculate the final score

In [8]:

#funzione forse inutile
def sum_vectors_score(dense_score, sparse_score):
    final_score = []
    for i in range(len(dense_score)):
        row = []
        for j in range(len(dense_score[i])):
            row.append(dense_score[i][j] + sparse_score[i][j])
        final_score.append(row)
    return final_score


final_score = sum_vectors_score(dense_score_results,sparse_score_results)
final_score = np.stack(final_score)
final_score 
 
    
    
    
def table_final_score(final_score):
    df = pd.DataFrame(final_score)

    df.index = ["Query " + str(i+1) for i in range(len(df.index))]
    df.columns = ["Document " + str(i+1) for i in range(len(df.columns))]

    return df

df = table_final_score(final_score)
df
    

,Document 1,Document 2,Document 3,Document 4,Document 5,Document 6,Document 7,Document 8,Document 9,Document 10,Document 11
Query 1,0.266100,0.377585,0.269571,0.473410,0.603978,0.341799,0.493245,0.435917,0.136205,0.373188,0.266100
Query 2,0.324503,0.219681,0.169559,0.296075,0.505359,0.200209,0.481966,0.348569,0.281434,0.228287,0.324503
Query 3,0.153645,0.119676,0.285546,0.139068,0.416846,0.187099,0.356040,0.207124,0.195489,0.120588,0.153645
Query 4,0.166681,0.175452,0.157920,0.226517,0.266544,0.214995,0.428582,0.259010,0.168591,0.175452,0.166681
Query 5,0.290604,0.205079,0.105684,0.275287,0.378720,0.325200,0.261496,0.183062,0.194384,0.205948,0.290604
Query 6,0.166459,0.209536,0.207605,0.192798,0.254442,0.159554,0.256521,0.227439,0.267136,0.203719,0.166459
Query 7,0.250586,0.194642,0.244185,0.220003,0.361672,0.188566,0.444623,0.243737,0.371042,0.195603,0.250586
Query 8,0.336789,0.355880,0.253950,0.406531,0.426588,0.289356,0.355287,0.303702,0.409178,0.335245,0.336789


# Tok-k documents 

In [9]:
def found_top_k(final_score, k):
    top_k = []
    for scores in final_score:
        sorted_scores = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)[:k]
        top_k.append([(index+1, score) for index, score in sorted_scores])
    return top_k


list_top_k = found_top_k(final_score,3) 
list_top_k_sparse = found_top_k(sparse_score_results,2)
list_top_k_dense = found_top_k(dense_score_results,2)


In [10]:
results_k = table_final_score(list_top_k)
print("TOP-K FINAL: ")
#results_k

TOP-K FINAL: 


In [11]:
results_k_sparse = table_final_score(list_top_k_sparse)
print("TOP-K SPARSE: ")
#results_k_sparse

TOP-K SPARSE: 


In [12]:
results_k_dense = table_final_score(list_top_k_dense)
print("TOP-K DENSE: ")
#results_k_dense

TOP-K DENSE: 


In [13]:
def printDocsQuery(documents,queries,results_k,list_top_k):  
    for query in queries:   
        print("QUERY: "+ query[1])
        print()
        elem = int(query[0])-1
        doc = [a for a, b in list_top_k[elem]]
        for position in doc:
            #print(documents[position][0])
            print(position)
            print()
    

#printDocsQuery(documents,queries,results_k,list_top_k)    

# Calculate Top-k first

In [14]:
def merge_lists(list1, list2):
    
    unique_set = set(list1)
    merged_list = list(unique_set)
    
    return merged_list


#CREO UNA LISTA DI LISTE CHE CONTIENE GLI INDICI.
#FACCIO L'UNIONE 
#CONFRONTO CON GLI INDICE DEL FINAL SCORE
#FRAZIONE : ELEMENTI IN COMUNE /k
    
def calculate_top_k_first(list_score, k_first,list_top_k):
    
    list_top_k_first_sparse = found_top_k(sparse_score_results,k_first)
    list_top_k_first_dense = found_top_k(dense_score_results,k_first)
    
    union_list_top_k_first = merge_lists(list_top_k_first_sparse,list_top_k_first_dense)   
    
    return union_list_top_k_first
 

#calculate_top_k_first(list_top_k, 2,list_top_k)

# --------------------------------------------------------

#  PARTIAMO DA QUI

# Calculate the finale score

In [15]:
import numpy as np


def calculate_full_score(sparse_score_results,dense_score_results):
    sparse_vector = np.stack(sparse_score_results)
    dense_vector = np.stack(dense_score_results)
    full_vector = sparse_vector + dense_vector
    
    return full_vector, dense_vector,sparse_vector
    
full_vector, dense_vector,sparse_vector = calculate_full_score(sparse_score_results,dense_score_results)

# Choose the query that you want analiyze

In [16]:

def choose_query(n):
    score_sparse = sparse_vector [n]
    score_dense = dense_vector [n]
    score_full = full_vector [n]
    
    return score_sparse, score_dense,score_full

score_sparse, score_dense,score_full = choose_query(0)    

In [17]:
K = 4
K_p = 4


In [18]:
def ground_truth_tok_k(K):
    ground_truth = np.argsort(score_full)[::-1][:K]
    
    return ground_truth 

ground_truth = ground_truth_tok_k(K)
ground_truth


array([4, 6, 3, 7], dtype=int64)

# For sparse and dense vector we take the best k_p elements

In [19]:
def best_top_k_first(K_p,score_dense,score_sparse):
    
    best_dense = np.argsort(score_dense)[::-1][:K_p]
    best_sparse = np.argsort(score_sparse)[::-1][:K_p]
    
    return best_sparse, best_dense


best_sparse, best_dense = best_top_k_first(K_p,score_dense,score_sparse)    

# Union, without duplicate, of best sparse and best dense elements of vectors

In [36]:
def union_best_vectors(best_sparse, best_dense):
    best_index_score = list(set(list(best_sparse) + list(best_dense))) #union
    return best_index_score

best_index_score = union_best_vectors(best_sparse, best_dense)

[1, 2, 3, 4, 6, 7, 9]

# Take the best score from the final score list using the indexes of sparse and dense union list and also return the best indexes sorted

In [21]:
def best_full_score(best_index_score):
    
    best_full_score = score_full[best_index_score]
    order_best_full_score = np.argsort(best_full_score)[::-1]  #questo lo ordino per ordinare i miei best
        
    return best_full_score, order_best_full_score

best_full_score, order_best_full_score = best_full_score(best_index_score)

order_best_full_score

array([3, 4, 2, 5, 0, 6, 1], dtype=int64)

In [22]:
best_full_score

array([0.37758475, 0.26957123, 0.47341027, 0.60397772, 0.49324487,
       0.43591741, 0.37318842])

In [41]:

actual_order = np.array(best_full_score)[list(order_best_full_score)]
actual_order

array([0.60397772, 0.49324487, 0.47341027, 0.43591741, 0.37758475,
       0.37318842, 0.26957123])

In [31]:
retrieved = actual_order[:K]
retrieved

array([4, 6, 3, 7], dtype=int64)

In [25]:
relevant = set(retrieved).intersection(ground_truth)
relevant

{3, 4, 6, 7}

In [26]:
len(relevant)/len(ground_truth)  # recall 

1.0